# The notebook used to generate the submission

Load data


In [52]:
import csv
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error
from verstack.stratified_continuous_split import scsplit # pip install verstack
import nltk# cant't find the stopword corpus otherwise
nltk.download('stopwords')
from nltk.corpus import stopwords 

import matplotlib.pyplot as plt



# Load the training data
train_data = pd.read_csv("data/train.csv")

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antoi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data cleaning


In [53]:
# add new features

train_data['url_count'] = train_data['urls'].str.count('http')
eval_data['url_count'] = eval_data['urls'].str.count('http')

#add followers/friends
train_data['followers_friends'] = train_data['followers_count'] / \
        (train_data['friends_count'].apply(lambda x: x+1))
eval_data['followers_friends'] = eval_data['followers_count'] / \
        (eval_data['friends_count'].apply(lambda x: x+1))  

drop_cols = ["mentions", "TweetID", "urls", "hashtags", "text", "verified"]

ids = eval_data["TweetID"]

train_data = train_data.drop(drop_cols, axis=1)
eval_data = eval_data.drop(drop_cols, axis=1)

In [54]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(max_features=0.93, max_leaf_nodes=2310, n_estimators=90, n_jobs=-1)

# We fit our model using the training data
X, y = train_data.drop(columns=["retweets_count"], axis=1), train_data["retweets_count"]
regressor.fit(X, y)

# And then we predict the values for our testing set
y_pred = regressor.predict(eval_data)

# Restore ids for writing to file
eval_data["TweetID"] = ids

In [55]:
def write_predictions_to_file(file_name, eval_data, predictions):
    with open(file_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["TweetID", "retweets_count"])
        for index, prediction in enumerate(predictions):
            writer.writerow([str(eval_data['TweetID'].iloc[index]) , str(int(prediction))])

In [56]:
write_predictions_to_file("submission.csv", eval_data, y_pred)